# Imports and dependencies

In [1]:
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# OpenAI key

In [2]:
load_dotenv()


True

In [15]:
# Use ChatOpenAI for chat-based models like GPT-4
client = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Prompt

In [8]:
prompt2 = "I need help categorizing a sentence into one of the following categories: Ciberseguridad, Enlace, Impresion, Licencias, Microinformatica, Movil, Nube, Servidores, Soporte or Atm.The sentence is the following: SERVICIO MONITOREO PREVENCION FRAUDES RAPPI"

# Component

In [16]:
# Make the API call
chat_completion = client.invoke([
    {"role": "user", "content": prompt2}
])


In [18]:
print(chat_completion.content) 


Ciberseguridad


# Multiple sentences

In [88]:
ruta_data_set = 'F:\DOCUMENTS\Xinergy\Proyecto glosas\LangChain\Project1\Glosas_Test_100.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='Sheet1')
# Select only the desired columns
df = df[['Glosa', 'Categoria M2']]

In [46]:
#df2 = df.sample(10)

In [47]:
#df3 = df2.copy()

In [89]:
df3 = df.copy()

In [90]:
df3

,Glosa,Categoria M2
0,SERVICIOS PARA LA ADMINISTRACIÓN Y SOPORTE DE ATM,Atm
1,SERVICIO IMPLEMENTACIÓN SEGURIDAD IPS,Ciberseguridad
2,SERV EXTER SUBCONTRATADO SEGURIDAD TECNOLOGICA...,Ciberseguridad
3,ANEXO 12 PHISHING TASK FORCE,Ciberseguridad
4,SERV MONITOREO PREV FRAUDES,Ciberseguridad
...,...,...
95,SERVICIO DE MANTENCIÓN REMOTA PARA ROBOTS,Soporte
96,REPARACIONES EN DATA CENTER – GG,Soporte
97,SERVICIOS DE SOPORTE Y MESA DE AYUDA (AFECTO A...,Soporte
98,MANTENCIÓN DEL DATA CENTER – AA,Soporte


In [91]:
categories = ["Ciberseguridad", "Enlace", "Impresion", "Licencias", "Microinformatica", 
              "Movil", "Nube", "Servidores", "Soporte", "Atm"]

In [92]:
# Initialize LangChain OpenAI client
client = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [94]:
# Define a function to query OpenAI API for categorization
def categorize_sentence(sentence, categories):
    # Build the prompt
    prompt = f"""
        You are a categorization assistant. Classify the following sentence into one of these categories: {', '.join(categories)}. 
        Only choose a category from the provided list. You have to categorize each sentence into one of the defined categories.

        Sentence: {sentence}

        Category:
    """

    # Make the API call using LangChain's `invoke()`
    try:
        chat_completion = client.invoke([
            {"role": "user", "content": prompt}
        ])
        
        # ✅ Extract response using `.content`
        return chat_completion.content.strip()
    
    except Exception as e:
        return f"Error: {str(e)}"

In [95]:
# Apply the function to the "Glosa" column and store results in a new column
df3['Predicted_Category'] = df3['Glosa'].apply(lambda sentence: categorize_sentence(sentence, categories))

In [96]:
df3

,Glosa,Categoria M2,Predicted_Category
0,SERVICIOS PARA LA ADMINISTRACIÓN Y SOPORTE DE ATM,Atm,Soporte
1,SERVICIO IMPLEMENTACIÓN SEGURIDAD IPS,Ciberseguridad,Ciberseguridad
2,SERV EXTER SUBCONTRATADO SEGURIDAD TECNOLOGICA...,Ciberseguridad,Servidores
3,ANEXO 12 PHISHING TASK FORCE,Ciberseguridad,Ciberseguridad
4,SERV MONITOREO PREV FRAUDES,Ciberseguridad,Ciberseguridad
...,...,...,...
95,SERVICIO DE MANTENCIÓN REMOTA PARA ROBOTS,Soporte,Servidores
96,REPARACIONES EN DATA CENTER – GG,Soporte,Servidores
97,SERVICIOS DE SOPORTE Y MESA DE AYUDA (AFECTO A...,Soporte,Soporte
98,MANTENCIÓN DEL DATA CENTER – AA,Soporte,Servidores


In [97]:
df3.to_excel(r'F:\DOCUMENTS\Xinergy\Proyecto glosas\results_Test_100_Categorized.xlsx', index=False)
	